In [1]:
# Uploading libraries
import ast
import pandas as pd
import seaborn as sns
import numpy as np
from datasets import load_dataset
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from adjustText import adjust_text


# Uploading dataset
df_2019_q4 = pd.read_csv('Original_data_Q1_2019_Q1_2020/Divvy_Trips_2019_Q4.csv')


In [2]:
df_2019_q4.sample(10)

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
44695,25270423,2019-10-04 06:51:24,2019-10-04 06:56:24,860,300.0,192,Canal St & Adams St,51,Clark St & Randolph St,Subscriber,Male,1962.0
623060,25877985,2019-12-13 08:20:26,2019-12-13 08:27:05,6362,399.0,331,Halsted St & Clybourn Ave,48,Larrabee St & Kingsbury St,Subscriber,Male,1990.0
35560,25260764,2019-10-03 14:56:54,2019-10-03 15:28:18,3080,"1,883.0",117,Wilton Ave & Belmont Ave,35,Streeter Dr & Grand Ave,Customer,NaN,NaN
504245,25751882,2019-11-21 18:02:17,2019-11-21 18:09:55,67,457.0,432,Clark St & Lunt Ave,660,Sheridan Rd & Columbia Ave,Subscriber,Male,1977.0
171712,25402920,2019-10-13 12:24:26,2019-10-13 12:28:57,880,270.0,119,Ashland Ave & Lake St,285,Wood St & Hubbard St,Subscriber,Female,1977.0
139980,25369826,2019-10-10 11:00:14,2019-10-10 11:23:31,1198,"1,396.0",113,Bissell St & Armitage Ave,145,Mies van der Rohe Way & Chestnut St,Customer,NaN,NaN
110950,25339699,2019-10-08 17:06:06,2019-10-08 17:17:49,3697,702.0,51,Clark St & Randolph St,186,Ogden Ave & Race Ave,Subscriber,Male,1986.0
626539,25881683,2019-12-13 16:28:48,2019-12-13 16:43:26,2884,878.0,287,Franklin St & Monroe St,172,Rush St & Cedar St,Subscriber,Male,1983.0
149229,25379492,2019-10-10 20:21:05,2019-10-10 20:35:30,6036,864.0,176,Clark St & Elm St,28,Larrabee St & Menomonee St,Subscriber,Female,1989.0
484349,25730887,2019-11-19 08:20:51,2019-11-19 08:30:55,2224,603.0,192,Canal St & Adams St,125,Rush St & Hubbard St,Subscriber,Male,1983.0


In [3]:
df_2019_q4['usertype'] = df_2019_q4['usertype'].replace('Customer', 'casual')
df_2019_q4['usertype'] = df_2019_q4['usertype'].replace('Subscriber', 'member')

In [4]:
columns_to_drop = ["tripduration", "from_station_id", "from_station_name", "to_station_id", "to_station_name", "gender", "birthyear", "bikeid"]
df_clean_2019_q4 = df_2019_q4.drop(columns=columns_to_drop)

df_clean_2019_q4.sample(10)

,trip_id,start_time,end_time,usertype
654571,25911028,2019-12-19 15:06:38,2019-12-19 15:19:07,member
173084,25404356,2019-10-13 13:51:27,2019-10-13 14:23:41,casual
105793,25334359,2019-10-08 12:00:28,2019-10-08 12:08:53,member
163117,25393887,2019-10-12 15:00:45,2019-10-12 15:20:43,casual
161101,25391776,2019-10-12 12:17:48,2019-10-12 12:48:02,member
427975,25671541,2019-11-07 19:13:11,2019-11-07 19:22:19,member
18592,25243110,2019-10-02 05:03:03,2019-10-02 05:22:12,member
547566,25797992,2019-11-30 06:40:59,2019-11-30 06:43:50,member
230771,25463909,2019-10-17 22:22:06,2019-10-17 22:38:01,member
665098,25922034,2019-12-21 13:47:27,2019-12-21 13:54:59,member


In [5]:
# Formatted columns start_time & end_time from Series to date&time then created new column where calculated trip duration

df_clean_2019_q4['start_time'] = pd.to_datetime(df_clean_2019_q4['start_time'], format='%Y-%m-%d %H:%M:%S')
df_clean_2019_q4['end_time'] = pd.to_datetime(df_clean_2019_q4['end_time'], format='%Y-%m-%d %H:%M:%S')


df_clean_2019_q4['ride_length_seconds'] = (df_clean_2019_q4['end_time'] - df_clean_2019_q4['start_time']).dt.total_seconds()

df_clean_2019_q4['ride_length_seconds'] = df_clean_2019_q4['ride_length_seconds'].astype(int)


def seconds_to_hhmmss(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


df_clean_2019_q4['ride_length'] = df_clean_2019_q4['ride_length_seconds'].apply(seconds_to_hhmmss)


df_clean_2019_q4.sample(10)

,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length
298315,25534411,2019-10-23 14:27:03,2019-10-23 14:55:09,casual,1686,00:28:06
240882,25474419,2019-10-18 17:00:16,2019-10-18 17:07:06,member,410,00:06:50
663229,25920100,2019-12-20 23:06:08,2019-12-20 23:18:15,member,727,00:12:07
131528,25361051,2019-10-09 18:26:58,2019-10-09 18:43:14,member,976,00:16:16
183351,25414918,2019-10-14 13:50:19,2019-10-14 14:13:28,casual,1389,00:23:09
510764,25758793,2019-11-22 16:27:06,2019-11-22 16:39:59,member,773,00:12:53
384184,25625520,2019-11-02 22:07:13,2019-11-02 22:26:41,member,1168,00:19:28
228766,25461811,2019-10-17 18:29:12,2019-10-17 18:36:04,member,412,00:06:52
172658,25403909,2019-10-13 13:24:23,2019-10-13 13:31:19,member,416,00:06:56
445582,25689825,2019-11-11 06:34:29,2019-11-11 06:47:35,member,786,00:13:06


In [6]:
df_clean_2019_q4['day_of_week'] = df_clean_2019_q4['start_time'].dt.day_name()
df_clean_2019_q4.sample(10)

,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length,day_of_week
155217,25385696,2019-10-11 16:21:00,2019-10-11 16:26:55,member,355,00:05:55,Friday
426103,25669589,2019-11-07 16:50:46,2019-11-07 16:55:18,member,272,00:04:32,Thursday
227839,25460864,2019-10-17 17:49:17,2019-10-17 18:10:51,member,1294,00:21:34,Thursday
566954,25818605,2019-12-04 08:07:23,2019-12-04 08:23:18,member,955,00:15:55,Wednesday
355234,25594263,2019-10-29 07:50:05,2019-10-29 07:54:20,member,255,00:04:15,Tuesday
120279,25349322,2019-10-09 08:20:06,2019-10-09 08:42:39,member,1353,00:22:33,Wednesday
24278,25249030,2019-10-02 16:45:41,2019-10-02 17:03:47,member,1086,00:18:06,Wednesday
305480,25541883,2019-10-24 05:44:14,2019-10-24 05:52:35,casual,501,00:08:21,Thursday
17990,25242492,2019-10-01 22:09:15,2019-10-01 22:34:40,casual,1525,00:25:25,Tuesday
474720,25720780,2019-11-18 04:57:59,2019-11-18 05:02:04,member,245,00:04:05,Monday


In [7]:
df_clean_2019_q4.to_csv('Cleaned_2019_Q4.csv', index=False)